In [1]:
import numpy as np
import pandas as pd
import datetime
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils import plot_model
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
seed = 0
np.random.seed(seed)

In [3]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [5]:
df_train = pd.read_csv('../Dataset/train_ver2.csv/train_ver2.csv')

E:\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df_test = pd.read_csv('../Dataset/test_ver2.csv/test_ver2.csv')

E:\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
personal_col = df_train.columns[:24]
product_col = df_train.columns[24:]
print(personal_col)
print(product_col)

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')
Index(['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')


In [8]:
pd.set_option('display.max_columns', None)

### Data 설명
- fecha_dato             : 날짜
- ncodpers               : 고객 식별번호
- ind_empleado           : 고용 지표 (A: active, B: ex employed, F: filial, N: not employee, P: passive)
- pais_residencia        : 거주국가
- sexo                   : 성별
- age                    : 나이
- fecha_alta             : 은행과 첫 계약을 체결한 날짜
- ind_nuevo              : 신규 고객 지표(6개월 이내 신규 고객일 경우 값 = 1)
- antiguedad             : 은행 거래 누적 기간(월)
- indrel                 : 고객 등급(1 : 1등급, 99: 해당 달에 고객 1등급이 해제되는 1등급 고객)
- ult_fec_cli_1t         : 1등급 고객으로서 마지막 날짜
- indrel_1mes            : 월초 기준 고객 등급(1: 1등급고개, 2: co-owner, P: potential, 3: former primary, 4 former co-owner)
- tiprel_1mes            : 월초 기준 고객 관계 유형 (A : active, I: inactive, P:former customer, R: potential)
- indresi                : 거주 지표(고객의 거주 국가와 은행이 위치한 국가 동일 여부 : S(yes) or N(No)
- indext                 : 외국인 지표(고객의 태어난 국가와 은행이 위치한 국가 동일 여부 : S(yes) or N(No)
- conyuemp               : 배우자 지표(1 : 은행 직원을 배우자로 둔 고객)
- canal_entrada          : 고객 유입 채널
- indfall                : 고객 사망 여부(S (yes) or N(No))
- tipodom                : 주소 유형(1: primary address)
- cod_prov               : 지방 코드(주소 기반)
- nomprov                : 지방 이름 
- ind_actividad_cliente  : 활발성 지표(1: active customer, 2: inactive customer)
- renta                  : 가구 총수입
- segmento               : 분류(01: VIP, 02: 개인, 03: 대졸)
- ind_ahor_fin_ult1      : 예금
- ind_aval_fin_ult1      : 보증
- ind_cco_fin_ult1       : 당좌 예금
- ind_cder_fin_ult1      : 파생 상품 계좌
- ind_cno_fin_ult1       : 급여 계정
- ind_ctju_fin_ult1      : 청소년 계정
- ind_ctma_fin_ult1      : 마스 특별 계정
- ind_ctop_fin_ult1      : 특정 계정
- ind_ctpp_fin_ult1      : 특정 플러스 계정
- ind_deco_fin_ult1      : 단기 예금
- ind_deme_fin_ult1      : 중기 예금
- ind_dela_fin_ult1      : 장기 예금
- ind_ecue_fin_ult1      : e-계정
- ind_fond_fin_ult1      : 펀드
- ind_hip_fin_ult1       : 부동산 대출
- ind_plan_fin_ult1      : 연금
- ind_pres_fin_ult1      : 대출
- ind_reca_fin_ult1      : 세금
- ind_tjcr_fin_ult1      : 신용카드
- ind_valo_fin_ult1      : 증권
- ind_viv_fin_ult1       : 홈 계정
- ind_nomina_ult1        : 급여
- ind_nom_pens_ult1      : 연금
- ind_recibo_ult1        : 직불 카드

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
fecha_dato               object
ncodpers                 int64
ind_empleado             object
pais_residencia          object
sexo                     object
age                      object
fecha_alta               object
ind_nuevo                float64
antiguedad               object
indrel                   float64
ult_fec_cli_1t           object
indrel_1mes              object
tiprel_1mes              object
indresi                  object
indext                   object
conyuemp                 object
canal_entrada            object
indfall                  object
tipodom                  float64
cod_prov                 float64
nomprov                  object
ind_actividad_cliente    float64
renta                    float64
segmento                 object
ind_ahor_fin_ult1        int64
ind_aval_fin_ult1        int64
ind_cco_fin_ult1         int64
ind_cder_fin_ult1  

In [16]:
num_cols = [col for col in df_train.columns[0:24] if df_train[col].dtype in ['float64', 'int64']]
df_train[num_cols].describe()

,ncodpers,ind_nuevo,indrel,tipodom,cod_prov,ind_actividad_cliente,renta
count,1.364731e+07,1.361958e+07,1.361958e+07,13619574.0,1.355372e+07,1.361958e+07,1.085293e+07
mean,8.349042e+05,5.956184e-02,1.178399e+00,1.0,2.657147e+01,4.578105e-01,1.342543e+05
std,4.315650e+05,2.366733e-01,4.177469e+00,0.0,1.278402e+01,4.982169e-01,2.306202e+05
min,1.588900e+04,0.000000e+00,1.000000e+00,1.0,1.000000e+00,0.000000e+00,1.202730e+03
25%,4.528130e+05,0.000000e+00,1.000000e+00,1.0,1.500000e+01,0.000000e+00,6.871098e+04
50%,9.318930e+05,0.000000e+00,1.000000e+00,1.0,2.800000e+01,0.000000e+00,1.018500e+05
75%,1.199286e+06,0.000000e+00,1.000000e+00,1.0,3.500000e+01,1.000000e+00,1.559560e+05
max,1.553689e+06,1.000000e+00,9.900000e+01,1.0,5.200000e+01,1.000000e+00,2.889440e+07


In [17]:
cat_cols = [col for col in df_train.columns[0:24] if df_train[col].dtype in ['object']]
df_train[cat_cols].describe()

,fecha_dato,ind_empleado,pais_residencia,sexo,age,fecha_alta,antiguedad,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,segmento
count,13647309,13619575,13619575,13619505,13647309,13619575,13647309,24793,13497528.0,13497528,13619575,13619575,1808,13461183,13619575,13553718,13457941
unique,17,5,118,2,235,6756,507,223,13.0,5,2,2,2,162,2,52,3
top,2016-05-28,N,ES,V,23,2014-07-28,0,2015-12-24,1.0,I,S,N,N,KHE,N,MADRID,02 - PARTICULARES
freq,931453,13610977,13553710,7424252,542682,57389,134335,763,7277607.0,7304875,13553711,12974839,1791,4055270,13584813,4409600,7960220


In [21]:
df_train[num_cols].head()

,ncodpers,ind_nuevo,indrel,tipodom,cod_prov,ind_actividad_cliente,renta
0,1375586,0.0,1.0,1.0,29.0,1.0,87218.10
1,1050611,0.0,1.0,1.0,13.0,0.0,35548.74
2,1050612,0.0,1.0,1.0,13.0,0.0,122179.11
3,1050613,0.0,1.0,1.0,50.0,0.0,119775.54
4,1050614,0.0,1.0,1.0,50.0,1.0,NaN


In [22]:
df_train[cat_cols].head()

,fecha_dato,ind_empleado,pais_residencia,sexo,age,fecha_alta,antiguedad,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,segmento
0,2015-01-28,N,ES,H,35,2015-01-12,6,NaN,1,A,S,N,NaN,KHL,N,MALAGA,02 - PARTICULARES
1,2015-01-28,N,ES,V,23,2012-08-10,35,NaN,1,I,S,S,NaN,KHE,N,CIUDAD REAL,03 - UNIVERSITARIO
2,2015-01-28,N,ES,V,23,2012-08-10,35,NaN,1,I,S,N,NaN,KHE,N,CIUDAD REAL,03 - UNIVERSITARIO
3,2015-01-28,N,ES,H,22,2012-08-10,35,NaN,1,I,S,N,NaN,KHD,N,ZARAGOZA,03 - UNIVERSITARIO
4,2015-01-28,N,ES,V,23,2012-08-10,35,NaN,1,A,S,N,NaN,KHE,N,ZARAGOZA,03 - UNIVERSITARIO


### 불필요한 컬럼 제거
- ind_nuevo              : 신규 고객 지표(6개월 이내 신규 고객일 경우 값 = 1)
- indresi                : 거주 지표(고객의 거주 국가와 은행이 위치한 국가 동일 여부 : S(yes) or N(No)
- indrel                 : 고객 등급(1 : 1등급, 99: 해당 달에 고객 1등급이 해제되는 1등급 고객)
- indfall                : 고객 사망 여부(S (yes) or N(No))
- tipodom                : 주소 유형(1: primary address)
- ind_empleado           : 고용 지표 (A: active, B: ex employed, F: filial, N: not employee, P: passive)
- pais_residencia        : 거주국가
- indrel_1mes            : 월초 기준 고객 등급(1: 1등급고개, 2: co-owner, P: potential, 3: former primary, 4 former co-owner)
- indext                 : 외국인 지표(고객의 태어난 국가와 은행이 위치한 국가 동일 여부 : S(yes) or N(No)
- conyuemp               : 배우자 지표(1 : 은행 직원을 배우자로 둔 고객)
- fecha_alta             : 은행과 첫 계약을 체결한 날짜
- tiprel_1mes            : 월초 기준 고객 관계 유형 (A : active, I: inactive, P:former customer, R: potential)

In [20]:
drop_column = ['ind_nuevo', 'indrel', 'indresi', 'indfall', 'tipodom', 'ind_empleado',
               'pais_residencia', 'indrel_1mes', 'indext','conyuemp','fecha_alta','tiprel_1mes']

In [ ]:
# 컬럼 제거 안함 (추후 성능 확인 후)

In [50]:
renta_df = df_test['renta'].copy()

In [51]:
renta_df.head()

0      326124.90
1             NA
2             NA
3      148402.98
4      106885.80
Name: renta, dtype: object

In [55]:
renta_df.fillna(-99, inplace=True)

In [63]:
(df_train.isnull().sum()/13647309.0)*100

fecha_dato                0.000000
ncodpers                  0.000000
ind_empleado              0.203220
pais_residencia           0.203220
sexo                      0.203732
age                       0.000000
fecha_alta                0.203220
ind_nuevo                 0.203220
antiguedad                0.000000
indrel                    0.203220
ult_fec_cli_1t           99.818330
indrel_1mes               1.097513
tiprel_1mes               1.097513
indresi                   0.203220
indext                    0.203220
conyuemp                 99.986752
canal_entrada             1.363829
indfall                   0.203220
tipodom                   0.203227
cod_prov                  0.685784
nomprov                   0.685784
ind_actividad_cliente     0.203220
renta                    20.475648
segmento                  1.387585
ind_ahor_fin_ult1         0.000000
ind_aval_fin_ult1         0.000000
ind_cco_fin_ult1          0.000000
ind_cder_fin_ult1         0.000000
ind_cno_fin_ult1    

In [74]:
renta_df.replace('         NA', -99, inplace=True)

In [78]:
renta_df = renta_df.astype(float)

In [80]:
renta_df.head()

0    326124.90
1       -99.00
2       -99.00
3    148402.98
4    106885.80
Name: renta, dtype: float64

In [83]:
renta_df.astype(np.int64)

0         326124
1            -99
2            -99
3         148402
4         106885
           ...  
929610    128643
929611       -99
929612     72765
929613    147488
929614       -99
Name: renta, Length: 929615, dtype: int64